In [0]:
# 텐서플로우 이하에서 사용 
import tensorflow as tf
from tensorflow import keras

%tensorflow_version 1.x

In [0]:
from tensorflow.keras import models, layers, backend
from tensorflow.keras.datasets import mnist  # datasets

# 데이터 준비 

In [0]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [0]:
img_rows, img_cols = X_train.shape[1:] # 이미지 크기 
img_rows, img_cols   # 28, 28

In [0]:
# 데이터의 포멧에 따라, 채널의 위치를 조정
backend.image_data_format()
# 'channels_last', 'channels_first'

In [0]:
# 데이터를 이런 형식으로 맟춰서 준비 

input_channel = 1  # 채널 => 흑백이미지(1), 칼라(3) => (60000, 28, 28, 3)

if backend.image_data_format == 'channels_last':
    # (1, rows, cols) => (1, H, W)
    # X_train.reshape(빈자리알아서(-1),채널,img_rows, img_cols)
    X_train = X_train.reshape(-1, 1, img_rows, img_cols)    # (60000, 1, 28, 28)
    X_test  = X_test.reshape( -1, 1, img_rows, img_cols )   # (10000, 1, 28, 28)
    input_shape = ( input_channel, img_rows, img_cols )
    
else :  # 'channels_last'
    X_train = X_train.reshape(-1, img_rows, img_cols, 1)
    X_test  = X_test.reshape(-1, img_rows, img_cols, 1)
    input_shape = ( img_rows, img_cols ,input_channel )

In [0]:
X_train.shape, X_test.shape

In [0]:
# 데이터 성분, 범위, 타입
X_train.dtype

In [0]:
# 타입 변경 
X_train = X_train.astype('float32')
X_test  = X_test.astype('float32')
X_train.dtype

In [0]:
# 정규화
# float32로 변화 처리하지만, 의미는 다르다
y_train[:10]

In [0]:
# 텐서플로우에서 사용한 방식으로 사용해 본다면 
# 원핫인코딩 스타일로 작성해 본다면 케라스의 유틸리티를 사용 
# 답을 구성하는 클래스의 총수( 가짓수, 답안의 케이스 수)
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_train[:10]

In [0]:

y_test = keras.utils.to_categorical(y_test, num_classes)

## 정규화, 표준화 
---


- 기본적인 튜토리얼 데이터 셋은 각 픽셀값이 0과 1로 정규화 되어있다. 
- 본데이터는 상위데이터이기 때문에 불러온 데이터들은 정규화 과정을 거치지 않은 상태이다. 이파일은 픽셀이 0과 255사이의 값을 가진다. 그렇기 때문에 전체 픽셀값을 255로 나누어서 정규화를 진행해야한다.-> 더욱 빠르게 연산이 가능하다.


In [0]:
# 정규화 처리(최대값 기준)
X_train /= 255
X_test  /= 255

In [0]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

# 신경망구성

In [0]:
# 인공신경망을 만든다고 선언적 표현, 모델 생성
model = models.Sequential()

In [0]:
# 합성곱층 생성 추가
# filters : 출력 채널수 -> 1을 넣어서 32가 출력
# kernal_size : 3x3
model.add( layers.Conv2D(32, 
                         kernel_size=(3,3),
                         strides=(1, 1),
                         padding='valid',
                         activation='relu',
                         input_shape=input_shape
                         ) )

In [0]:
# 풀링층 생성 추가 
model.add(layers.MaxPool2D(pool_size=(2,2),))

In [0]:
# 과적합 방지 추가 
model.add(layers.Dropout(0.25))

In [0]:
# 합성곱층 생성 추가 : 기본값들이 있는 부분들은 생략해서 표현
model.add(layers.Conv2D(64, 
                         kernel_size=(3,3),
                         #strides=(1, 1),
                         #padding='valid',
                         activation='relu',
                         #input_shape=input_shape,
                         ))

In [0]:
# 풀링층 생성 추가
model.add( layers.MaxPool2D( pool_size=(2,2) ) )

In [0]:
# 과적합 방지 추가
model.add( layers.Dropout(0.25) )

In [0]:
# 전결합층 추가 
model.add(layers.Dense(0.5))

In [0]:
# 출력층 추가 => y
# num_classes => 10
model.add( layers.Dense( num_classes, activation='softmax' ) )

In [0]:
# 손실함수, 최적화, 정확도 추가 혹은 컴파일
model.compile( loss=keras.losses.categorical_crossentropy,
               optimizer='rmsprop',
               metrics=['accuracy']
              )

# 훈련 및 테스트

In [0]:
# 실험값, 임계값
# 실험 횟수, 세대수, 훈련 횟수
epochs     = 10 # 설정값
# 1회 학습시 사용되는 데이터양
batch_size = 128 # 설정값

In [0]:
model.fit( X_train, y_train, 
          epochs     = epochs,
          batch_size = batch_size,
          validation_split=0.2          
          )

In [0]:
# 테스트 데이터로 확인
score = model.evaluate(X_test, y_test)
score